In [2]:
import pandas as pd

### 1. Data import

In [3]:
df_schirmer2003 = pd.read_csv('./Output/Schirmer2003/Output.csv')

In [5]:
df_schirmer2003.tail()

,Entry,Gene names
39,Q8N163,Ccar2
40,Q6NW34,Nepro
41,Q969X5,Ergic1
42,Q8NC56,Lemd2
43,Q5GFL6,Vwa2


In [6]:
df_korfali2010 = pd.read_csv('./Output/Korfali_2010.csv')

In [8]:
df_korfali2010.head()

,Entry_Korfali_2010,Gene_name_obtained
0,Q9UH99,SUN2
1,P42166,TMPO
2,Q14739,LBR
3,Q8TEM1,NUP210
4,Q9BTV4,TMEM43


In [7]:
df_wilkie2010 = pd.read_csv('./Output/Wilkie_2010.csv')

In [10]:
df_wilkie2010.head()

,Entry_Wilkie,Gene_name_obtained
0,Q9UH99,SUN2
1,Q5JTV8,TOR1AIP1
2,Q14739,LBR
3,O94901,SUN1
4,Q8TEM1,NUP210


In [8]:
df_korfali2012 = pd.read_csv('./Output/Korfali2012/Korfali2012_Hs.csv')

In [6]:
df_korfali2012.head()

,Unnamed: 0,tissue,gene name,alternate names,accession numbers,NE:MM ratio by dNSAF,reference,Uniprot_id
0,0,liver enriched,TMEM53,"NET4, transmembrane protein 53",ref|NP_081113.1|,2.57,"This study and Schirmer, E.C., et al. (2003). ...",Q6P2H8
1,1,liver enriched,TMEM120A,"NET29, transmembrane protein induced by tumor ...",ref|NP_766129.1|,INF,"This study and Malik, P., et al. (2010) Cell M...",Q9BXJ8
2,2,liver enriched,SCARA5,"NET33, PREDICTED: similar to protease, serine,...",gi|109502608|ref|XP_001066668.1|,0.1,"This study and Malik, P., et al. (2010) Cell M...",Q6ZMJ2
3,3,liver enriched,TMEM74,"NET36, PREDICTED: hypothetical protein [Rattus...",ref|XP_001063530.1|,3.36,"This study and Malik, P., et al. (2010) Cell M...",Q96NL1
4,4,liver enriched,PPAPDC3,"NET39, phosphatidic acid phosphatase type 2 do...",gi|59891419|ref|NP_001012349.1|;gi|34147436|re...,4.42,"This study and Schirmer, E.C., et al. (2003). ...",Q8NBV4


In [9]:
df_cheng2019 = pd.read_csv('./Output/Cheng2019.csv')

In [11]:
df_cheng2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gene_name             242 non-null    object 
 1   entry_h               242 non-null    object 
 2   NE Enrich Score in U  242 non-null    float64
 3   NE Enrich Score in A  242 non-null    float64
 4   NE Enrich Score in M  242 non-null    float64
dtypes: float64(3), object(2)
memory usage: 9.6+ KB


### 2. Merge

### 2-1. Schirmer + Korfali2010

In [12]:
df = df_schirmer2003.merge(df_korfali2010, left_on='Entry', right_on='Entry_Korfali_2010', how='outer')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NCBI ID             56 non-null     object
 1   Entry               56 non-null     object
 2   Entry name          56 non-null     object
 3   Protein names       56 non-null     object
 4   Gene names          53 non-null     object
 5   Organism            56 non-null     object
 6   Entry_Korfali_2010  139 non-null    object
 7   Gene_name_obtained  139 non-null    object
dtypes: object(8)
memory usage: 12.7+ KB


In [13]:
df['UniprotID'] = df.Entry.fillna(df.Entry_Korfali_2010)
df['GeneName'] = df['Gene names'].fillna(df.Gene_name_obtained)

In [106]:
df.columns

Index(['NCBI ID', 'Entry', 'Entry name', 'Protein names', 'Gene names',
       'Organism', 'Entry_Korfali_2010', 'Gene_name_obtained', 'UniprotID',
       'GeneName'],
      dtype='object')

In [14]:
df = df[['UniprotID', 'GeneName', 'Entry', 'Entry_Korfali_2010']]

df.fillna(0, inplace=True)

df.Entry = df.Entry.apply(lambda x: 1 if x != 0 else 0)
df.Entry_Korfali_2010 = df.Entry_Korfali_2010.apply(lambda x: 1 if x != 0 else 0)

df = df.rename(columns={'Entry':'Schirmer_2003', 'Entry_Korfali_2010':'Korfali_2010'})

In [15]:
df.tail()

,UniprotID,GeneName,Schirmer_2003,Korfali_2010
152,Q86V85,GPR180,0,1
153,Q8N386,LRRC25,0,1
154,Q5TGY1,TMCO4,0,1
155,Q9NX61,TMEM161A,0,1
156,Q96AA3,RFT1,0,1


### 2-2. Add Wilkie 2010

In [16]:
df = df.merge(df_wilkie2010, left_on='UniprotID', right_on='Entry_Wilkie', how='outer')

df.UniprotID = df.UniprotID.fillna(df.Entry_Wilkie)
df.GeneName = df.GeneName.fillna(df.Gene_name_obtained)

df.fillna(0, inplace=True)

In [17]:
df.tail()

,UniprotID,GeneName,Schirmer_2003,Korfali_2010,Entry_Wilkie,Gene_name_obtained
182,Q6ZV29,PNPLA7,0.0,0.0,Q6ZV29,PNPLA7
183,Q0P6H9,TMEM62,0.0,0.0,Q0P6H9,TMEM62
184,Q8IZF2,ADGRF5,0.0,0.0,Q8IZF2,ADGRF5
185,A6NMS7,LRRC37A,0.0,0.0,A6NMS7,LRRC37A
186,Q7Z407,CSMD3,0.0,0.0,Q7Z407,CSMD3


In [18]:
df.Entry_Wilkie = df.Entry_Wilkie.apply(lambda x: 1 if x != 0 else 0)

In [19]:
df = df.drop(columns=['Gene_name_obtained'], axis=1)
df = df.rename(columns={'Entry_Wilkie': 'Wilkie_2010'})

### 2-3. Add Korfali 2012

In [20]:
df = df.merge(df_korfali2012, left_on='UniprotID', right_on='Uniprot_id', how='outer')

In [21]:
df.UniprotID = df.UniprotID.fillna(df.Uniprot_id)
df.GeneName = df.GeneName.fillna(df['gene name'])

df.fillna(0, inplace=True)

df.Uniprot_id = df.Uniprot_id.apply(lambda x: 1 if x != 0 else 0)

In [22]:
df.columns

Index(['UniprotID', 'GeneName', 'Schirmer_2003', 'Korfali_2010', 'Wilkie_2010',
       'Unnamed: 0', 'tissue', 'gene name', 'alternate names',
       'accession numbers', 'NE:MM ratio by dNSAF', 'reference', 'Uniprot_id'],
      dtype='object')

In [23]:
df = df[['UniprotID', 'GeneName', 'Schirmer_2003', 'Korfali_2010', 'Wilkie_2010',
           'Uniprot_id']]
df = df.rename(columns={'Uniprot_id':'Korfali_2012'})

In [24]:
df.tail()

,UniprotID,GeneName,Schirmer_2003,Korfali_2010,Wilkie_2010,Korfali_2012
212,Q96PC5,CTAGE5,0.0,0.0,0.0,1
213,Q8WUY1,C8orf55,0.0,0.0,0.0,1
214,Q7Z2K6,ERMP1,0.0,0.0,0.0,1
215,Q7Z3C6,ATG9A,0.0,0.0,0.0,1
216,Q6DD88,ATLA3,0.0,0.0,0.0,1


### 2-4. Add Cheng 2019

In [25]:
df = df.merge(df_cheng2019, left_on='UniprotID', right_on='entry_h', how='outer')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413 entries, 0 to 412
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   UniprotID             218 non-null    object 
 1   GeneName              218 non-null    object 
 2   Schirmer_2003         218 non-null    float64
 3   Korfali_2010          218 non-null    float64
 4   Wilkie_2010           218 non-null    float64
 5   Korfali_2012          218 non-null    float64
 6   gene_name             243 non-null    object 
 7   entry_h               243 non-null    object 
 8   NE Enrich Score in U  243 non-null    float64
 9   NE Enrich Score in A  243 non-null    float64
 10  NE Enrich Score in M  243 non-null    float64
dtypes: float64(7), object(4)
memory usage: 38.7+ KB


In [29]:
df.UniprotID = df.UniprotID.fillna(df.entry_h)
df.GeneName = df.GeneName.fillna(df.gene_name)

df.fillna(0, inplace=True)

df.entry_h = df.entry_h.apply(lambda x: 1 if x != 0 else 0)

In [30]:
df.columns

Index(['UniprotID', 'GeneName', 'Schirmer_2003', 'Korfali_2010', 'Wilkie_2010',
       'Korfali_2012', 'gene_name', 'entry_h', 'NE Enrich Score in U',
       'NE Enrich Score in A', 'NE Enrich Score in M'],
      dtype='object')

In [31]:
df = df.rename(columns={'entry_h':'Cheng_2019'})
df = df.drop(columns=['gene_name'], axis=1)

In [34]:
df.head()

,UniprotID,GeneName,Schirmer_2003,Korfali_2010,Wilkie_2010,Korfali_2012,Cheng_2019,NE Enrich Score in U,NE Enrich Score in A,NE Enrich Score in M
0,Q92604,LPGAT1 FAM34A KIAA0205,1.0,1.0,1.0,1.0,0,0.0,0.0,0.0
1,Q9Y3T9,NOC2L NIR,1.0,1.0,1.0,0.0,0,0.0,0.0,0.0
2,Q9Y3T9,Noc2l,1.0,1.0,1.0,0.0,0,0.0,0.0,0.0
3,Q8NBX0,SCCPDH CGI-49,1.0,1.0,1.0,1.0,0,0.0,0.0,0.0
4,A0A384NPM7,SCCPDH hCG_1782151,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0
